In [1]:
%cd mmdetection

#완디비 설치
!pip install -q --upgrade wandb
!pip3 install openmim
!mim install mmcv-full
!pip install -e .

/aiffel/aiffel/Aimmo/mmdetection
     |████████████████████████████████| 50 kB 4.3 MB/s             
     |████████████████████████████████| 237 kB 11.2 MB/s            
     |████████████████████████████████| 51 kB 12.5 MB/s            
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
     |████████████████████████████████| 49.7 MB 4.8 MB/s             
     |████████████████████████████████| 190 kB 6.0 MB/s            
Obtaining file:///aiffel/aiffel/Aimmo/mmdetection
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0.6-cp39-cp39-linux_x86_64.whl size=104366 sha256=e9092b8b93932c529ca70023a59daad533498d15c51afa5e2ec78ca51b221b3e
  Stored in directory: /aiffel/.cache/pip/wheels/2f/58/25/e78f1f766e904a9071266661d20d0bc6644df86bcd160aba11
Successfully built pycocotools


In [2]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset,CustomDataset
from mmdet.apis import set_random_seed
import os.path as osp
import os
import numpy as np

import wandb
print(wandb.__version__)

/opt/conda/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


0.13.6


In [3]:
# # 완디비 로그인
wandb.login(key= '')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

In [4]:
wandb.init(project="test-project", entity="yolo_x")

wandb: Currently logged in as: cychuckpark (yolo_x). Use `wandb login --relogin` to force relogin


In [5]:
@DATASETS.register_module(force=True)
class Aimmo(CustomDataset):
    CLASSES = ('car', 'bus', 'truck','pedestrian')
    def load_annotations(self, ann_file):
        cat2label = {k:i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
        data_infos = []
    
        for image_id in image_list:
            filename = os.path.splitext(image_id)[0]#os.path.splitext('{0:}/{1:}'.format(img_prefix, image_id))[0]
            #image = cv2.imread(filename)
            data_info = {'filename': filename,
                         'width': 1920, 
                         'height': 1024}
            label_prefix = self.img_prefix.replace('image', 'middle_txt')
            lines = mmcv.list_from_file(osp.join(label_prefix, str(image_id)))

            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [ [float(info) for info in x[1:5]] for x in content]

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []

            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    gt_labels.append(cat2label[bbox_name])
                else:
                    gt_bboxes_ignore.append(bbox)
                    gt_labels_ignore.append(-1)
            
            data_anno = {
              'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
              'labels': np.array(gt_labels, dtype=np.compat.long),
              'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
              'labels_ignore': np.array(gt_labels_ignore, dtype=np.compat.long)
            }
            data_info.update(ann=data_anno)
            data_infos.append(data_info)
        return data_infos

In [6]:
# !wget -c https://download.openmmlab.com/mmdetection/v2.0/fcos/fcos_center-normbbox-centeronreg-giou_r50_caffe_fpn_gn-head_dcn_1x_coco/fcos_center-normbbox-centeronreg-giou_r50_caffe_fpn_gn-head_dcn_1x_coco-ae4d8b3d.pth \
#       -O checkpoints/fcos_center-normbbox-centeronreg-giou_r50_caffe_fpn_gn-head_dcn_1x_coco-ae4d8b3d.pth


In [7]:
config_file = 'configs/fcos/fcos_center-normbbox-centeronreg-giou_r50_caffe_fpn_gn-head_dcn_1x_coco.py'
checkpoint_file = 'fcos_center-normbbox-centeronreg-giou_r50_caffe_fpn_gn-head_dcn_1x_coco-ae4d8b3d.pth'
cfg = Config.fromfile(config_file)

cfg.load_from = 'checkpoints/fcos_center-normbbox-centeronreg-giou_r50_caffe_fpn_gn-head_dcn_1x_coco-ae4d8b3d.pth'
cfg.work_dir = 'fcos'
cfg.resume_from = 'fcos/epoch_7.pth'



cfg.dataset_type = 'Aimmo'
cfg.data_root = 'data/'

cfg.train_pipeline[2]['img_scale'] = (960,512)
cfg.test_pipeline[1]['img_scale'] = (960,512)



cfg.data.train.type = 'Aimmo'
cfg.data.train.data_root = 'data/'
cfg.data.train.ann_file = 'middle_train.txt'
cfg.data.train.img_prefix = 'image'

cfg.data.train.pipeline[2]['img_scale'] = (960,512)


cfg.data.val.type = 'Aimmo'
cfg.data.val.data_root = 'data/'
cfg.data.val.ann_file = 'middle_val.txt'
cfg.data.val.img_prefix = 'image'

cfg.data.val.pipeline[1]['img_scale'] = (960,512)

cfg.data.test.type = 'Aimmo'
cfg.data.test.data_root = 'data/'
cfg.data.test.ann_file = 'middle_test.txt'
cfg.data.test.img_prefix = 'image'

cfg.data.test.pipeline[1]['img_scale'] = (960,512)

cfg.lr_config.policy='step'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'
cfg.evaluation.metric='mAP'


cfg.checkpoint_config.interval = 1
cfg.log_config.interval = 100
cfg.evaluation.interval = 1
cfg.runner['max_epochs'] = 15

cfg.model.bbox_head['num_classes'] =4

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 0

print(cfg.pretty_text)

dataset_type = 'Aimmo'
data_root = 'data/'
img_norm_cfg = dict(
    mean=[103.53, 116.28, 123.675], std=[1.0, 1.0, 1.0], to_rgb=False)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(960, 512), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[103.53, 116.28, 123.675],
        std=[1.0, 1.0, 1.0],
        to_rgb=False),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(960, 512),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
                type='Normalize',
                mean=[103.53, 116.28, 123.675],
                std=[1.

In [8]:
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={'project': 'test-project', 'entity' : 'yolo_x', 'name' : 'fcos'})] 

In [9]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

/aiffel/aiffel/Aimmo/mmdetection/mmdet/datasets/custom.py:181: UserWarning: CustomDataset does not support filtering empty gt images.
  warnings.warn(
2022-12-11 05:57:17,425 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-12-11 05:57:20,177 - mmdet - INFO - load checkpoint from local path: fcos/epoch_7.pth
2022-12-11 05:57:23,111 - root - INFO - ModulatedDeformConvPack backbone.layer2.0.conv2 is upgraded to version 2.
2022-12-11 05:57:23,113 - root - INFO - ModulatedDeformConvPack backbone.layer2.1.conv2 is upgraded to version 2.
2022-12-11 05:57:23,116 - root - INFO - ModulatedDeformConvPack backbone.layer2.2.conv2 is upgraded to version 2.
2022-12-11 05:57:23,119 - root - INFO - ModulatedDeformConvPack backbone.layer2.3.conv2 is upgraded to version 2.
2022-12-11 05:57:23,120 - root - INFO - ModulatedDeformConvPack backbone.layer3.0.conv2 is upgraded to version 2.
2022-12-11 05:57:23,123 - root - INFO - ModulatedDeformConvPack backbone.layer3.1.conv2 

2022-12-11 05:58:31,488 - mmdet - INFO - Epoch [8][100/10452]	lr: 1.000e-02, eta: 14:07:23, time: 0.609, data_time: 0.272, memory: 1638, loss_cls: 0.1040, loss_bbox: 0.1424, loss_centerness: 0.6085, loss: 0.8549
2022-12-11 05:58:31,488 - mmdet - INFO - Epoch [8][100/10452]	lr: 1.000e-02, eta: 14:07:23, time: 0.609, data_time: 0.272, memory: 1638, loss_cls: 0.1040, loss_bbox: 0.1424, loss_centerness: 0.6085, loss: 0.8549
2022-12-11 05:59:28,354 - mmdet - INFO - Epoch [8][200/10452]	lr: 1.000e-02, eta: 13:38:25, time: 0.569, data_time: 0.230, memory: 1652, loss_cls: 0.1035, loss_bbox: 0.1428, loss_centerness: 0.6132, loss: 0.8595
2022-12-11 05:59:28,354 - mmdet - INFO - Epoch [8][200/10452]	lr: 1.000e-02, eta: 13:38:25, time: 0.569, data_time: 0.230, memory: 1652, loss_cls: 0.1035, loss_bbox: 0.1428, loss_centerness: 0.6132, loss: 0.8595
2022-12-11 06:00:26,746 - mmdet - INFO - Epoch [8][300/10452]	lr: 1.000e-02, eta: 13:35:14, time: 0.584, data_time: 0.245, memory: 1652, loss_cls: 0.104

2022-12-11 06:16:43,725 - mmdet - INFO - Epoch [8][2000/10452]	lr: 1.000e-02, eta: 13:04:15, time: 0.577, data_time: 0.237, memory: 1672, loss_cls: 0.1092, loss_bbox: 0.1457, loss_centerness: 0.6125, loss: 0.8674
2022-12-11 06:17:41,145 - mmdet - INFO - Epoch [8][2100/10452]	lr: 1.000e-02, eta: 13:03:08, time: 0.574, data_time: 0.234, memory: 1672, loss_cls: 0.1069, loss_bbox: 0.1472, loss_centerness: 0.6127, loss: 0.8669
2022-12-11 06:17:41,145 - mmdet - INFO - Epoch [8][2100/10452]	lr: 1.000e-02, eta: 13:03:08, time: 0.574, data_time: 0.234, memory: 1672, loss_cls: 0.1069, loss_bbox: 0.1472, loss_centerness: 0.6127, loss: 0.8669
2022-12-11 06:18:38,539 - mmdet - INFO - Epoch [8][2200/10452]	lr: 1.000e-02, eta: 13:02:01, time: 0.574, data_time: 0.235, memory: 1672, loss_cls: 0.1120, loss_bbox: 0.1388, loss_centerness: 0.6082, loss: 0.8591
2022-12-11 06:18:38,539 - mmdet - INFO - Epoch [8][2200/10452]	lr: 1.000e-02, eta: 13:02:01, time: 0.574, data_time: 0.235, memory: 1672, loss_cls: 

2022-12-11 06:36:01,350 - mmdet - INFO - Epoch [8][4000/10452]	lr: 1.000e-02, eta: 12:46:32, time: 0.583, data_time: 0.243, memory: 1672, loss_cls: 0.1069, loss_bbox: 0.1366, loss_centerness: 0.6096, loss: 0.8531
2022-12-11 06:36:01,350 - mmdet - INFO - Epoch [8][4000/10452]	lr: 1.000e-02, eta: 12:46:32, time: 0.583, data_time: 0.243, memory: 1672, loss_cls: 0.1069, loss_bbox: 0.1366, loss_centerness: 0.6096, loss: 0.8531
2022-12-11 06:36:59,034 - mmdet - INFO - Epoch [8][4100/10452]	lr: 1.000e-02, eta: 12:45:32, time: 0.577, data_time: 0.238, memory: 1672, loss_cls: 0.1007, loss_bbox: 0.1398, loss_centerness: 0.6112, loss: 0.8517
2022-12-11 06:36:59,034 - mmdet - INFO - Epoch [8][4100/10452]	lr: 1.000e-02, eta: 12:45:32, time: 0.577, data_time: 0.238, memory: 1672, loss_cls: 0.1007, loss_bbox: 0.1398, loss_centerness: 0.6112, loss: 0.8517
2022-12-11 06:37:56,690 - mmdet - INFO - Epoch [8][4200/10452]	lr: 1.000e-02, eta: 12:44:33, time: 0.577, data_time: 0.237, memory: 1672, loss_cls: 

2022-12-11 06:54:16,283 - mmdet - INFO - Epoch [8][5900/10452]	lr: 1.000e-02, eta: 12:27:39, time: 0.577, data_time: 0.238, memory: 1672, loss_cls: 0.1048, loss_bbox: 0.1453, loss_centerness: 0.6119, loss: 0.8620
2022-12-11 06:55:14,275 - mmdet - INFO - Epoch [8][6000/10452]	lr: 1.000e-02, eta: 12:26:45, time: 0.580, data_time: 0.241, memory: 1672, loss_cls: 0.1095, loss_bbox: 0.1405, loss_centerness: 0.6093, loss: 0.8593
2022-12-11 06:55:14,275 - mmdet - INFO - Epoch [8][6000/10452]	lr: 1.000e-02, eta: 12:26:45, time: 0.580, data_time: 0.241, memory: 1672, loss_cls: 0.1095, loss_bbox: 0.1405, loss_centerness: 0.6093, loss: 0.8593
2022-12-11 06:56:12,468 - mmdet - INFO - Epoch [8][6100/10452]	lr: 1.000e-02, eta: 12:25:53, time: 0.582, data_time: 0.242, memory: 1672, loss_cls: 0.1072, loss_bbox: 0.1440, loss_centerness: 0.6115, loss: 0.8628
2022-12-11 06:56:12,468 - mmdet - INFO - Epoch [8][6100/10452]	lr: 1.000e-02, eta: 12:25:53, time: 0.582, data_time: 0.242, memory: 1672, loss_cls: 

...

2022-12-11 20:01:22,203 - mmdet - INFO - Epoch [15][7600/10452]	lr: 1.000e-04, eta: 0:27:47, time: 0.586, data_time: 0.247, memory: 1675, loss_cls: 0.0632, loss_bbox: 0.1037, loss_centerness: 0.6056, loss: 0.7724
2022-12-11 20:01:22,203 - mmdet - INFO - Epoch [15][7600/10452]	lr: 1.000e-04, eta: 0:27:47, time: 0.586, data_time: 0.247, memory: 1675, loss_cls: 0.0632, loss_bbox: 0.1037, loss_centerness: 0.6056, loss: 0.7724
2022-12-11 20:02:21,565 - mmdet - INFO - Epoch [15][7700/10452]	lr: 1.000e-04, eta: 0:26:49, time: 0.594, data_time: 0.254, memory: 1675, loss_cls: 0.0673, loss_bbox: 0.1017, loss_centerness: 0.6055, loss: 0.7745
2022-12-11 20:02:21,565 - mmdet - INFO - Epoch [15][7700/10452]	lr: 1.000e-04, eta: 0:26:49, time: 0.594, data_time: 0.254, memory: 1675, loss_cls: 0.0673, loss_bbox: 0.1017, loss_centerness: 0.6055, loss: 0.7745
2022-12-11 20:03:19,773 - mmdet - INFO - Epoch [15][7800/10452]	lr: 1.000e-04, eta: 0:25:50, time: 0.582, data_time: 0.242, memory: 1675, loss_cls: 

2022-12-11 20:19:58,213 - mmdet - INFO - Epoch [15][9500/10452]	lr: 1.000e-04, eta: 0:09:16, time: 0.579, data_time: 0.240, memory: 1675, loss_cls: 0.0718, loss_bbox: 0.1108, loss_centerness: 0.6064, loss: 0.7890
2022-12-11 20:20:56,722 - mmdet - INFO - Epoch [15][9600/10452]	lr: 1.000e-04, eta: 0:08:18, time: 0.585, data_time: 0.245, memory: 1675, loss_cls: 0.0707, loss_bbox: 0.1065, loss_centerness: 0.6062, loss: 0.7834
2022-12-11 20:20:56,722 - mmdet - INFO - Epoch [15][9600/10452]	lr: 1.000e-04, eta: 0:08:18, time: 0.585, data_time: 0.245, memory: 1675, loss_cls: 0.0707, loss_bbox: 0.1065, loss_centerness: 0.6062, loss: 0.7834
2022-12-11 20:21:54,710 - mmdet - INFO - Epoch [15][9700/10452]	lr: 1.000e-04, eta: 0:07:19, time: 0.580, data_time: 0.240, memory: 1675, loss_cls: 0.0703, loss_bbox: 0.1048, loss_centerness: 0.6058, loss: 0.7809
2022-12-11 20:21:54,710 - mmdet - INFO - Epoch [15][9700/10452]	lr: 1.000e-04, eta: 0:07:19, time: 0.580, data_time: 0.240, memory: 1675, loss_cls: 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.3 task/s, elapsed: 442s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-11 20:36:36,810 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 68495 | 0.972  | 0.937 |
| bus        | 1786  | 18645 | 0.960  | 0.896 |
| truck      | 3084  | 41594 | 0.961  | 0.863 |
| pedestrian | 3063  | 36923 | 0.855  | 0.709 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.851 |
+------------+-------+-------+--------+-------+
2022-12-11 20:36:36,810 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 68495 | 0.972  | 0.937 |
| bus        | 1786  | 18645 | 0.960  | 0.896 |
| truck      | 3084  | 41594 | 0.961  | 0.863 |
| pedestrian | 3063  | 36923 | 0.855  | 0.709 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.851 |
+---

learning_rate,█████▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,███▆█▄▄▄▄▄▄▄▄▃▃▃▃▃▂▃▃▄▃▂▃▃▂▂▁▂▃▃▂▁▂▂▂▂▂▁
train/loss_bbox,███▆█▄▄▄▄▃▄▄▄▃▄▃▃▃▂▃▃▃▃▂▃▃▂▂▁▂▃▃▃▂▂▂▂▂▂▂
train/loss_centerness,▇▇▆▄█▄▂▃▂▄▄▃▃▃▄▂▂▃▃▂▃▄▆▂▃▂▂▂▁▃▃▂▃▂▂▁▂▂▁▁
train/loss_cls,███▇█▄▄▄▄▄▄▃▄▄▃▂▄▂▂▃▃▄▃▂▃▂▂▂▁▁▃▂▂▁▁▃▂▂▂▁
val/AP50,▁▇▇█████
val/mAP,▁▇▇█████
learning_rate,0.0001
momentum,0.9
train/loss,0.76812
